In [659]:
import pandas as pd

In [660]:
csv_1 = pd.read_csv('consensus.csv', index_col=0, sep='|', encoding='utf-8')

In [661]:

import ast
csv_1['hypothesis'] = csv_1['hypothesis'].apply(ast.literal_eval)
csv_1

,premise,hypothesis,rationales,split,correct_index,LLM_answer
0,"""There are 10 apples on an apple tree. Three ...","[park, coloring book, garden center, math prob...",1. The commonly agreed-upon answer to the ques...,1. the commonly agreed-upon answer to the ques...,{'math problem': 163},math problem
1,"A John is a bum. Much like the stereotype, he...","[bus depot, beach, train station, bridge, bridge]",2. The commonly agreed-upon answer to the ques...,2. the commonly agreed-upon answer to the ques...,{'bridge': 179},bridge
2,A bad person places little value on being hone...,"[excellent, upright, premium, competent, sincere]",3. The commonly agreed-upon answer to the ques...,3. the commonly agreed-upon answer to the ques...,{'sincere': 145},sincere
3,"A bald eagle flies over St. Paul, where is it?","[texas, thermal, minnesota, canada, photograph]",4. The commonly agreed-upon answer to the ques...,4. the commonly agreed-upon answer to the ques...,{'minnesota': 103},minnesota
4,A battleship is a powerful vessel. If you nee...,"[yatch, corvette, aircraft carrier, destroyer,...",5. The commonly agreed-upon answer to the ques...,5. the commonly agreed-upon answer to the ques...,{'destroyer': 155},destroyer
...,...,...,...,...,...,...
9736,where is the information superhighway?,"[heavily travelled area, cyberspace, industria...",7. The commonly agreed-upon answer to the ques...,7. the commonly agreed-upon answer to the ques...,{'cyberspace': 99},cyberspace
9737,where would a good newspaper boy put a paper i...,"[library, lawn, trash, roof, front door]",8. The commonly agreed-upon answer to the ques...,8. the commonly agreed-upon answer to the ques...,{'front door': 118},front door
9738,which door with lock regularly holds cars?,"[house, autolock, garage, file cabinet, safe]",9. The commonly agreed-upon answer to the ques...,9. the commonly agreed-upon answer to the ques...,{'garage': 103},garage
9739,which half of the earth do marmots live in?,"[south, northern hemisphere, north america, va...",10. The commonly agreed-upon answer to the que...,10. the commonly agreed-upon answer to the que...,"{'northern hemisphere': 105, 'north america': ...",northern hemisphere


In [662]:
# from datasets import load_dataset
# import pandas as pd

# dataset = load_dataset("esnli")
# train_df = pd.DataFrame(dataset['train']).sample(frac=1/100, random_state=0)
# validation = pd.DataFrame(dataset['validation']).sample(frac=1/100, random_state=0)

# train_df = pd.concat([train_df, validation], ignore_index=True)

In [663]:
prompt_template= '''
Questions: %s \n
Choices: (A) %s, (B) %s, (C) %s, (D) %s, (E) %s \
What is the commonly agreed-upon answer to the question '%s' with options %s, %s, %s, %s %s? Justify your answer based on general knowledge. \n
'''

In [664]:
error = csv_1[csv_1.rationales.isna()]

In [665]:
# error['prompt'] = error.apply(lambda x: prompt_template % (x['premise'], x['hypothesis'][0], x['hypothesis'][1], x['hypothesis'][2], x['hypothesis'][3], x['hypothesis'][4],
#                                                           x['premise'], x['hypothesis'][0], x['hypothesis'][1], x['hypothesis'][2], x['hypothesis'][3], x['hypothesis'][4]), axis=1)


In [666]:
error.index

Int64Index([], dtype='int64')

In [667]:
print(error.values)

[]


In [668]:
csv_1['split'] = csv_1.rationales.apply(lambda x: x.lower().split('most likely answer'))

In [669]:
csv_1.split = csv_1.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [670]:
after_handle = pd.read_csv('error.csv', index_col=0, sep='|', encoding='utf-8')

# replace the csv_1 with after_handle
csv_1.loc[after_handle.index, 'rationales'] = after_handle['rationales']

In [671]:
csv_1['correct_index'] = [{} for _ in range(len(csv_1))]
for index, row in csv_1.iterrows():
    for choice in row['hypothesis']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice)


In [672]:
csv_1[csv_1.LLM_answer.isna()]

for index, row in csv_1[csv_1.LLM_answer.isna()].iterrows():
    for choice in row['hypothesis']:
        if choice in row['rationales'].lower():
            row.loc['correct_index'][choice] = row['rationales'].lower().index(choice)

In [673]:
import numpy as np
csv_1['LLM_answer'] =csv_1.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [677]:
csv_1[csv_1.LLM_answer.isna()]

,premise,hypothesis,rationales,split,correct_index,LLM_answer


In [744]:
# csv_1[csv_1.LLM_answer == 'entailed'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'entails'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'contradicts'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradicted'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradictory'] = 'contradiction'

In [680]:
paper = pd.read_csv('paper.csv', index_col=0)

In [689]:
csv_1['question'] = csv_1.apply(lambda x: f'{x.premise}\nAnswer Choices:\n(a) {x.hypothesis[0]}\n(b) {x.hypothesis[1]}\n(x.hypothesis[ ]{x.hypothesis[2]}\n(d) {x.hypothesis[3]}\n(e) {x.hypothesis[4]}', axis=1)
csv_1.set_index('question', inplace=True)

In [715]:
paper['premise'] = paper.input.str.split('\n').apply(lambda x: x[0])
paper.set_index('premise', inplace=True)

In [718]:
total_df = csv_1[(csv_1.premise.isin(paper.premise))]
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"""There are 10 apples on an apple tree. Three fall off. Now there are X apples."" What is this an example of?\nAnswer Choices:\n(a) park\n(b) coloring book\n(x.hypothesis[ ]garden center\n(d) math problem\n(e) gravity","""There are 10 apples on an apple tree. Three ...","[park, coloring book, garden center, math prob...",1. The commonly agreed-upon answer to the ques...,1. the commonly agreed-upon answer to the ques...,{'math problem': 163},math problem
"A John is a bum. Much like the stereotype, he lives near this sort of transportation infrastructure. Where does he live?\nAnswer Choices:\n(a) bus depot\n(b) beach\n(x.hypothesis[ ]train station\n(d) bridge\n(e) bridge","A John is a bum. Much like the stereotype, he...","[bus depot, beach, train station, bridge, bridge]",2. The commonly agreed-upon answer to the ques...,2. the commonly agreed-upon answer to the ques...,{'bridge': 179},bridge
"A bad person places little value on being honest, acting without pretense or being what?\nAnswer Choices:\n(a) excellent\n(b) upright\n(x.hypothesis[ ]premium\n(d) competent\n(e) sincere",A bad person places little value on being hone...,"[excellent, upright, premium, competent, sincere]",3. The commonly agreed-upon answer to the ques...,3. the commonly agreed-upon answer to the ques...,{'sincere': 145},sincere
"A bald eagle flies over St. Paul, where is it?\nAnswer Choices:\n(a) texas\n(b) thermal\n(x.hypothesis[ ]minnesota\n(d) canada\n(e) photograph","A bald eagle flies over St. Paul, where is it?","[texas, thermal, minnesota, canada, photograph]",4. The commonly agreed-upon answer to the ques...,4. the commonly agreed-upon answer to the ques...,{'minnesota': 103},minnesota
"A battleship is a powerful vessel. If you need something similar but faster, what would you use?\nAnswer Choices:\n(a) yatch\n(b) corvette\n(x.hypothesis[ ]aircraft carrier\n(d) destroyer\n(e) patrol boat",A battleship is a powerful vessel. If you nee...,"[yatch, corvette, aircraft carrier, destroyer,...",5. The commonly agreed-upon answer to the ques...,5. the commonly agreed-upon answer to the ques...,{'destroyer': 155},destroyer
...,...,...,...,...,...,...
where is the information superhighway?\nAnswer Choices:\n(a) heavily travelled area\n(b) cyberspace\n(x.hypothesis[ ]industrialized country\n(d) on the internet.\n(e) computer network,where is the information superhighway?,"[heavily travelled area, cyberspace, industria...",7. The commonly agreed-upon answer to the ques...,7. the commonly agreed-upon answer to the ques...,{'cyberspace': 99},cyberspace
where would a good newspaper boy put a paper in the rain?\nAnswer Choices:\n(a) library\n(b) lawn\n(x.hypothesis[ ]trash\n(d) roof\n(e) front door,where would a good newspaper boy put a paper i...,"[library, lawn, trash, roof, front door]",8. The commonly agreed-upon answer to the ques...,8. the commonly agreed-upon answer to the ques...,{'front door': 118},front door
which door with lock regularly holds cars?\nAnswer Choices:\n(a) house\n(b) autolock\n(x.hypothesis[ ]garage\n(d) file cabinet\n(e) safe,which door with lock regularly holds cars?,"[house, autolock, garage, file cabinet, safe]",9. The commonly agreed-upon answer to the ques...,9. the commonly agreed-upon answer to the ques...,{'garage': 103},garage


In [724]:
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"""There are 10 apples on an apple tree. Three fall off. Now there are X apples."" What is this an example of?\nAnswer Choices:\n(a) park\n(b) coloring book\n(x.hypothesis[ ]garden center\n(d) math problem\n(e) gravity","""There are 10 apples on an apple tree. Three ...","[park, coloring book, garden center, math prob...",1. The commonly agreed-upon answer to the ques...,1. the commonly agreed-upon answer to the ques...,{'math problem': 163},math problem
"A John is a bum. Much like the stereotype, he lives near this sort of transportation infrastructure. Where does he live?\nAnswer Choices:\n(a) bus depot\n(b) beach\n(x.hypothesis[ ]train station\n(d) bridge\n(e) bridge","A John is a bum. Much like the stereotype, he...","[bus depot, beach, train station, bridge, bridge]",2. The commonly agreed-upon answer to the ques...,2. the commonly agreed-upon answer to the ques...,{'bridge': 179},bridge
"A bad person places little value on being honest, acting without pretense or being what?\nAnswer Choices:\n(a) excellent\n(b) upright\n(x.hypothesis[ ]premium\n(d) competent\n(e) sincere",A bad person places little value on being hone...,"[excellent, upright, premium, competent, sincere]",3. The commonly agreed-upon answer to the ques...,3. the commonly agreed-upon answer to the ques...,{'sincere': 145},sincere
"A bald eagle flies over St. Paul, where is it?\nAnswer Choices:\n(a) texas\n(b) thermal\n(x.hypothesis[ ]minnesota\n(d) canada\n(e) photograph","A bald eagle flies over St. Paul, where is it?","[texas, thermal, minnesota, canada, photograph]",4. The commonly agreed-upon answer to the ques...,4. the commonly agreed-upon answer to the ques...,{'minnesota': 103},minnesota
"A battleship is a powerful vessel. If you need something similar but faster, what would you use?\nAnswer Choices:\n(a) yatch\n(b) corvette\n(x.hypothesis[ ]aircraft carrier\n(d) destroyer\n(e) patrol boat",A battleship is a powerful vessel. If you nee...,"[yatch, corvette, aircraft carrier, destroyer,...",5. The commonly agreed-upon answer to the ques...,5. the commonly agreed-upon answer to the ques...,{'destroyer': 155},destroyer
...,...,...,...,...,...,...
where is the information superhighway?\nAnswer Choices:\n(a) heavily travelled area\n(b) cyberspace\n(x.hypothesis[ ]industrialized country\n(d) on the internet.\n(e) computer network,where is the information superhighway?,"[heavily travelled area, cyberspace, industria...",7. The commonly agreed-upon answer to the ques...,7. the commonly agreed-upon answer to the ques...,{'cyberspace': 99},cyberspace
where would a good newspaper boy put a paper in the rain?\nAnswer Choices:\n(a) library\n(b) lawn\n(x.hypothesis[ ]trash\n(d) roof\n(e) front door,where would a good newspaper boy put a paper i...,"[library, lawn, trash, roof, front door]",8. The commonly agreed-upon answer to the ques...,8. the commonly agreed-upon answer to the ques...,{'front door': 118},front door
which door with lock regularly holds cars?\nAnswer Choices:\n(a) house\n(b) autolock\n(x.hypothesis[ ]garage\n(d) file cabinet\n(e) safe,which door with lock regularly holds cars?,"[house, autolock, garage, file cabinet, safe]",9. The commonly agreed-upon answer to the ques...,9. the commonly agreed-upon answer to the ques...,{'garage': 103},garage


In [727]:
paper.set_index('premise', inplace=True)
total_df.set_index('premise', inplace=True)
paper['my_label'] = total_df['LLM_answer']

In [728]:
paper

,input,label,llm_label,rationale,my_label
premise,,,,,
"A kid is studying new words at home in the kitchen, where could a dictionary be?",A kid is studying new words at home in the kit...,table,library,The answer must be a place where a dictionary ...,shelf
Sarah did all of it. She was the best saleswoman in the city. But she was never acknowledge for her what?,Sarah did all of it. She was the best saleswo...,work hard,productivity,The answer must be something that Sarah did th...,productivity
Where do you bring your flight bag?,Where do you bring your flight bag?\nAnswer Ch...,airport,airport,The answer must be the place where you bring y...,airport
when im competing in a competition what should i aim for,when im competing in a competition what should...,wins,wins,The answer should be the goal of someone compe...,success
"The computer stopped working, even though John tried his best. What happened to it?","The computer stopped working, even though John...",crash,crash,The answer must be something that happened to ...,crash
...,...,...,...,...,...
"If people need something on the other side of a body of water, what must they do?",If people need something on the other side of ...,cross river,cross river,The answer must be something that people do to...,cross river
What kind of pain can be caused my watching television too much?,What kind of pain can be caused my watching te...,headaches,headaches,The answer must be a pain caused by watching t...,headaches
"The tourists swam in a swimming pool, isolated from the rest of the country, where were they?","The tourists swam in a swimming pool, isolated...",resort hotel,resort hotel,The answer must be a place where tourists can ...,resort hotel


In [729]:
sum(paper.label == paper.my_label) / len(paper)

0.7465352633199877

In [730]:
sum(paper.label == paper.llm_label) / len(paper)

0.7426342264654553

In [41]:
total_df.to_csv('consensus - full.csv', encoding='utf-8')